In [1]:
import os
#set your environment variable for SSL certificate
certi_path = r"C:\Users\bertr\AppData\Local\Programs\Python\Python39\Lib\site-packages\pip\_vendor\certifi\cacert.pem"
os.environ['REQUESTS_CA_BUNDLE'] = certi_path
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import random
import time
import json
import pandas as pd

In [2]:
def initialize_driver(headers_list, proxy_list):
    options = Options()
    #select a random user-agent from the list
    user_agent = random.choice(headers_list)["User-Agent"]
    options.add_argument(f"user-agent={user_agent}")
    
    #select a random proxy from the list
    proxy = random.choice(proxy_list)
    if proxy:
        options.add_argument(f"--proxy-server={proxy}")
    
    #add some common options
    #options.add_argument("--headless")
    options.add_argument("--disable-extensions")
    options.add_argument("--ignore-certificate-errors")

    #initialize Chrome WebDriver with the specified options
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    #set implicit wait of 10sec
    driver.implicitly_wait(10)

    return driver

In [3]:
def MainPage(driver, url):
    driver.get(url)
    time.sleep(3)

In [9]:
def nbHisytoryByPage(driver):
    try:
        liste_histoires = driver.find_element(By.ID, "liste_histoires")
        search_results = driver.find_elements(By.TAG_NAME, "article")
        search_results_count = len(search_results)
        return search_results_count

    except Exception as e:
        print("An error occurred in NB_OFFER:", str(e))
        return 0  # Or handle the exception as needed

In [7]:
def nbHisytoryTot(driver):
    try:
        search_results = driver.find_element(By.CSS_SELECTOR, 'div[class="row align-items-center gy-2 mb-4 pb-1 pb-sm-2 pb-lg-3"]')
        text_nb_hostory_tot = search_results.find_element(By.CSS_SELECTOR, 'h2[class="mb-lg-0"]').text.split('(')[1].split(')')[0]
        return int(text_nb_hostory_tot)

    except Exception as e:
        print("An error occurred in NB_OFFER:", str(e))
        return 0  # Or handle the exception as needed

In [11]:
def Click(driver, pos):
    '''Click on the link'''
    try:
        element = driver.find_elements(By.CSS_SELECTOR, 'a[class="btn btn-sm btn-primary mb-1"]')[pos]
        element.click()
        return 1

    except Exception as e:
        print("An error occurred in CLICK:", str(e))
        return 0

In [17]:
def ClickMoreHistory(driver):
    '''Click on the link'''
    try:
        element = driver.find_element(By.ID, "loadmorebtn")
        element.click()
        return 1

    except Exception as e:
        print("An error occurred in CLICK:", str(e))
        return 0

In [109]:
def GetDatas(driver, story):
    time.sleep(3)
    try:
        age = driver.find_elements(By.CSS_SELECTOR, 'span[class="badge border border-light text-light fs-sm mb-1"]')[1].text
        title = driver.find_element(By.CSS_SELECTOR, 'h1[class="display-2 mb-4"]').text
        history = driver.find_element(By.CSS_SELECTOR, 'div[class="col-lg-9 col-xl-8"]').find_element(By.CSS_SELECTOR, 'div[class="fs-lg"]').get_attribute("outerHTML")
        accordion_question = driver.find_element(By.ID, "collapseOne")
        question_histoire = []
        all_questions = accordion_question.find_elements(By.CSS_SELECTOR, 'h3[class="lead"]')
        for i in range(0, len(all_questions)):
            div_responses = accordion_question.find_elements(By.CSS_SELECTOR, 'div[class="card mb-4"]')[i]
            all_responses = div_responses.find_elements(By.CSS_SELECTOR, 'label[class="form-check-label"]')
            correction = driver.find_elements(By.CLASS_NAME, 'text-success')[i]
            tab_response = []
            for a in range(0, len(all_responses)):
                tab_response.append(all_responses[a].text)
            question_histoire.append({
            "question"+ str(i+1): {
                "enonce": accordion_question.find_elements(By.CSS_SELECTOR, 'h3[class="lead"]')[i].text,
                "responses": tab_response,
                "correction": correction.get_attribute("id").split('_')[2]
            }})
        accordion_glossaire = driver.find_element(By.ID, "collapseTwo").find_element(By.TAG_NAME, 'dl')
        mots = accordion_glossaire.find_elements(By.TAG_NAME, 'dt')
        defs = accordion_glossaire.find_elements(By.TAG_NAME, 'dd')
        dictionnaire_glossaire = {}
        for i in range(0, len(mots)):
            dictionnaire_glossaire[mots[i].get_attribute("textContent")] = defs[i].get_attribute("textContent")
        current_story = [age, title, history, question_histoire, dictionnaire_glossaire]
        story.append(current_story)
        f = open('history.txt', 'a')
        f.write(str(current_story))
        print(f'write job : {driver.current_url}')
        f.write('\n')
        f.close()
        return 1
    except Exception as e:
        print(f"Error HTML PARSING: {e}")

In [5]:
driver = initialize_driver([
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Host": "httpbin.org",
        "Sec-Ch-Ua": "\"Chromium\";v=\"92\", \" Not A;Brand\";v=\"99\", \"Google Chrome\";v=\"92\"",
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.5",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.5",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en;q=0.5",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0"
    },
    {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Host": "httpbin.org",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15"
    }
], [False])

In [6]:
MainPage(driver, r"https://www.meshistoiresdusoir.fr/g/histoires-fantastiques/")

In [120]:
nbHisytoryTotVar = nbHisytoryTot(driver)

In [119]:
nbHisytoryByPageVar = nbHisytoryByPage(driver)

In [12]:
Click(driver, 0)

1

In [117]:
ClickMoreHistory(driver)

1

In [118]:
nbHisytoryByPage(driver)

36

In [121]:
nb_page = nbHisytoryTotVar // nbHisytoryByPageVar
if nbHisytoryTotVar % nbHisytoryByPageVar != 0:
    nb_page = nb_page + 1

In [122]:
nb_page

39

In [ ]:
story = []
nb_history_page_init = nbHisytoryByPage(driver)
for a in range(0, nb_page - 1):
    nb_history_page = nbHisytoryByPage(driver)
    for i in range((a * nb_history_page_init), nb_history_page):
        Click(driver, (i)
        GetDatas(driver, story)
        driver.back()
    time.sleep(3)
    ClickMoreHistory(driver)

In [108]:
story = []
GetDatas(driver, story)

épreuves
0
Des difficultés ou des défis à surmonter.
épreuves
1
Une femme dotée de pouvoirs magiques, parfois considérée comme maléfique.
Sorcière
2
Personne qui excelle dans un domaine particulier.
Maître
3
Pièges ou obstacles qui rendent la progression difficile.
Embûches
4
La volonté de ne pas abandonner face à l'adversité.
Détermination
5
Le fait de s'unir avec d'autres pour atteindre un objectif commun.
Alliance
write job : https://www.meshistoiresdusoir.fr/h/les-protecteurs-de-lombre/


1